In [ ]:
!pip install fairlearn
!pip install xgboost
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f214d3dbfedf2a00dbf9407427514e7308da53408b922483aaf74474231a9734
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# loading in one-hot encoded, cleaned (floats), stratified subsample of x_train, y_train, y_test, y_train for implementing bias mitigation techniqes.
import sys
import csv
import time
import random
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice, combinations
import itertools
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, cohen_kappa_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio, demographic_parity_ratio
from fairlearn.metrics import demographic_parity_difference
from pyspark import SparkContext, SparkConf
if __name__ == '__main__':
  start_time = time.time()
  conf = SparkConf().setAppName('531_final_proj').setMaster('local[*]')
  sc = SparkContext.getOrCreate(conf=conf)
  sc.setSystemProperty('spark.driver.memory', '4g')
  sc.setSystemProperty('spark.executor.memory', '4g')
  sc.setLogLevel("ERROR")

In [ ]:
# Load the datasets
X_test = pd.read_csv('/content/X_test.csv', header=None, dtype=float)
X_train = pd.read_csv('/content/X_train.csv', header=None, dtype=float)
y_test = pd.read_csv('/content/y_test.csv', header=None, dtype=float)
y_train = pd.read_csv('/content/y_train.csv', header=None, dtype=float)

In [ ]:
#X_train_rdd.take(1)
lines = sc.textFile('/content/X_train.csv')
rdd = lines.map(lambda row: row.split(','))

In [ ]:
# make them into rdds now for later
X_test_rdd = sc.parallelize(X_test.values.tolist())
X_train_rdd = sc.parallelize(X_train.values.tolist())
y_test_rdd = sc.parallelize(y_test.values.tolist())
y_train_rdd = sc.parallelize(y_train.values.tolist())

In [ ]:
X_test_rdd.take(1)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

 # 1) Normal -- Without any bias mitigation or Smote

In [ ]:
params = {
    'objective': 'binary:hinge',
    'eval_metric': 'rmse',
}
xgb_model = xgb.XGBRegressor(**params)
xgb_model.fit(X_train, y_train)
preds = xgb_model.predict(X_test)

print("XGBoost Model RMSE : ", np.sqrt(mean_squared_error(y_test, preds)))
# Accuracy
accuracy = accuracy_score(y_test, preds)
print("XGBoost Model Accuracy:", accuracy)

XGBoost Model RMSE :  0.3246700446057128
XGBoost Model Accuracy: 0.8945893621357245


In [ ]:
# Stat Parity


gender_and_preds = X_test_rdd.map(lambda x: (x[5], x[6]))
gender_positive_rates = gender_and_preds.groupByKey().mapValues(lambda preds: sum(preds) / len(preds))


gender_positive_rates_collected = gender_positive_rates.collect()

#statistical parity difference
statistical_parity_difference = abs(gender_positive_rates_collected[0][1] - gender_positive_rates_collected[1][1])

print("Statistical Parity Difference (Gender):", statistical_parity_difference)
sens_data = X_test_rdd.map(lambda x: (x[5], x[6]))


# Collect the results
sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print(equalized_odds_difference(y_test,
                                preds,
                                sensitive_features=scalar_sens_data))

print(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data))
# Stat parity gender

sens_data = X_test_rdd.map(lambda x: (x[5], x[6]))  # Assuming gender is at index 4 and 5 in each data point

# Collect the results
sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print("Statistical Parity Ratio (Gender): ", demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data))


# Stat Parity -> medicare vs medicaid (insurance status)

sens_data = X_test_rdd.map(lambda x: (x[53:58]))  # Assuming gender is at index 4 and 5 in each data point

sens_data_collected = sens_data.collect()

scalar_sens_data = [status[0] for status in sens_data_collected]

print("Equalized Odds Ratio (Insurance Status)",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Insurance Status)", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))

# RACE: Stat Parity & equzlied odds -> race (11-17)
sens_data = X_test_rdd.map(lambda x: (x[11:18]))  # Assuming gender is at index 4 and 5 in each data point

# Collect the results
sens_data_collected = sens_data.collect()
scalar_sens_data = [race[0] for race in sens_data_collected]


print("Equalized Odds Ratio (Race):",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Race):", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))

# AGE
sens_data = X_test_rdd.map(lambda x: x[4])  # Assuming gender is at index 4 and 5 in each data point

# Collect the results
scalar_sens_data = sens_data.collect()
#scalar_sens_data = [age[0] for age in sens_data_collected]


print("Equalized Odds Ratio (Age):",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Age):", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))


#print("stat parity difference age:", (demographic_parity_difference(y_test, preds, sensitive_features=scalar_sens_data)))


## Results with SMOTE


In [ ]:
#sampling imbalance class with smote, smoteen
from imblearn.combine import SMOTEENN
import collections
counter = collections.Counter(y_train)
print('Before', counter)
# oversampling the train dataset using SMOTE + ENN
smenn = SMOTEENN()
X_train_smenn, y_train_smenn = smenn.fit_resample (X_train, y_train)
counter = collections.Counter (y_train_smenn)
print('After', counter)

In [ ]:

xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=3, n_jobs=-1, learning_rate=0.5, random_state=42)
xgb_model.fit(X_train_smenn, y_train_smenn)


preds = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test,preds)
kappa = cohen_kappa_score(y_test,preds)
f1 = f1_score(y_test, preds)
recall = recall_score(y_test, preds)
precision = precision_score(y_test, preds)


print("Results With Oversampling Smote + ENN")
print("XGBoost Accuracy:", accuracy)
print("Precision:", precision)

In [ ]:
print("SMOTE RESULTS:/n")

######  GENDER ######
# Equalized Odds
gender_and_preds = X_test_rdd.map(lambda x: (x[5], x[6]))
gender_positive_rates = gender_and_preds.groupByKey().mapValues(lambda preds: sum(preds) / len(preds))
gender_positive_rates_collected = gender_positive_rates.collect()
statistical_parity_difference = abs(gender_positive_rates_collected[0][1] - gender_positive_rates_collected[1][1])

print("Statistical Parity Difference (Gender):", statistical_parity_difference)
sens_data = X_test_rdd.map(lambda x: (x[5], x[6]))


sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print(equalized_odds_difference(y_test,
                                preds,
                                sensitive_features=scalar_sens_data))

print(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data))
# Stat parity gender with fairness learn metrics

sens_data = X_test_rdd.map(lambda x: (x[5], x[6]))
sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print("Statistical Parity Ratio (Gender): ", demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data))


# Stat Parity -> medicare vs medicaid (insurance status)

sens_data = X_test_rdd.map(lambda x: (x[53:58]))  # gender at index 4,5

sens_data_collected = sens_data.collect()

scalar_sens_data = [status[0] for status in sens_data_collected]

print("Equalized Odds Ratio (Insurance Status)",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Insurance Status)", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))

# RACE: Stat Parity & equzlied odds -> race (11-17)
sens_data = X_test_rdd.map(lambda x: (x[11:18]))  # Gender index 4, 5

# Collect the results
sens_data_collected = sens_data.collect()
scalar_sens_data = [race[0] for race in sens_data_collected]


print("Equalized Odds Ratio (Race):",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Race):", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))

# AGE
sens_data = X_test_rdd.map(lambda x: x[4])


scalar_sens_data = sens_data.collect()
#scalar_sens_data = [age[0] for age in sens_data_collected]


print("Equalized Odds Ratio (Age):",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("Statistical Parity Ratio (Age):", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))
